# Preamble

In [106]:
from IPython.display import display, Markdown

# Make connection to Slack

In [3]:
from private_config import SLACK_TOKEN

In [4]:
from slackclient import SlackClient

sc = SlackClient(SLACK_TOKEN)

In [5]:
channel_info = sc.api_call("channels.list")
channels = channel_info['channels']

In [6]:
[(k['name'], k['id']) for k in channels]

[(u'building_comments', u'C6GQ4GWMT'),
 (u'building-comments', u'C6GKKF7U2'),
 (u'cafe_comments', u'C6FCGM656'),
 (u'external_relations', u'C2CMBSDG8'),
 (u'general', u'C2CLS7MHC'),
 (u'movequestions', u'C3WHBE9JN'),
 (u'random', u'C2CM9H9BL'),
 (u'theatreclub', u'C2CLV66QG')]

In [ ]:
users = sc.api_call("users")

# Get messages of interest

(i.e. building_comments and cafe_comments from actual users)

In [7]:
included_channels = [u'building_comments', u'cafe_comments']
channel_ids = dict([(k['name'], k['id']) for k in channels if k['name'] in included_channels])
channel_ids

{u'building_comments': u'C6GQ4GWMT', u'cafe_comments': u'C6FCGM656'}

In [107]:
channel_comments = {}

for cid in channel_ids:
    print "grabbing history for %s..." % cid
    # grab the history for each
    results = sc.api_call("channels.history", channel=channel_ids[cid], count=1000)
    channel_comments[cid] = results['messages']

grabbing history for building_comments...
grabbing history for cafe_comments...


In [108]:
chat_messages = [msg for msg in reversed(channel_comments['cafe_comments']) if msg['type'] == 'message' and 'subtype' not in msg]

# Format and display messages

In [109]:
from itertools import groupby
from datetime import datetime
from pytz import timezone

In [110]:
def suffix(d):
    return 'th' if 11<=d<=13 else {1:'st',2:'nd',3:'rd'}.get(d%10, 'th')

def custom_strftime(format, t):
    return t.strftime(format).replace('{S}', str(t.day) + suffix(t.day))

print custom_strftime('%B {S}, %Y', datetime.now())

August 4th, 2017


In [111]:
formatted_chats = [
    {
        'datetime': (datetime
        .fromtimestamp(float(msg['ts']), timezone('UTC'))
        .astimezone(timezone('US/Eastern'))),
        'text': msg['text']
    }
    for msg in chat_messages
]

In [116]:
def escape_markdown(text):
    return text.replace("*", "\*")

In [120]:
["%s: %s" % (custom_strftime("%A, %B {S}, %-I:%M%p", msg['datetime']), msg['text']) for msg in formatted_chats]

[u'Monday, July 31st, 1:55PM: <@U58RD27SN> ',
 u'Monday, July 31st, 6:36PM: Hey ',
 u'Monday, July 31st, 7:02PM: <@U61K5FHQU> <@U58RD27SN> ',
 u'Monday, July 31st, 7:40PM: White bean soup was great',
 u'Monday, July 31st, 7:40PM: Fresh peach was not edible ',
 u'Monday, July 31st, 7:41PM: Double espresso was perfect',
 u'Tuesday, August 1st, 1:09PM: 2nd day in a row of perfect espresso',
 u'Tuesday, August 1st, 1:10PM: Studybot campus-survey',
 u'Tuesday, August 1st, 1:11PM: studybot campus-survey',
 u'Tuesday, August 1st, 2:26PM: Had a sandwich: turkey, mozzarella, all the veggies on multigrain.  It was very good.',
 u'Wednesday, August 2nd, 9:07AM: made to order omelette: (turkey, onions, and peppers): well-seasoned and perfectly cooked. (the _only_ thing is the cafe is still waiting on hot sauce..) but other than that, glowing reviews for breakfast!',
 u'Wednesday, August 2nd, 9:10AM: :+1::skin-tone-5:',
 u'Wednesday, August 2nd, 9:40AM: STARR should look into their credit card proc

In [132]:
grouped_messages = groupby(formatted_chats, lambda x: x['datetime'].strftime("%A, %B %-m"))
for date in (grouped_messages):
    print date[0], list(date[1])

Monday, July 7 [{'text': u'<@U58RD27SN> ', 'datetime': datetime.datetime(2017, 7, 31, 13, 55, 16, 28220, tzinfo=<DstTzInfo 'US/Eastern' EDT-1 day, 20:00:00 DST>)}, {'text': u'Hey ', 'datetime': datetime.datetime(2017, 7, 31, 18, 36, 3, 446092, tzinfo=<DstTzInfo 'US/Eastern' EDT-1 day, 20:00:00 DST>)}, {'text': u'<@U61K5FHQU> <@U58RD27SN> ', 'datetime': datetime.datetime(2017, 7, 31, 19, 2, 10, 920502, tzinfo=<DstTzInfo 'US/Eastern' EDT-1 day, 20:00:00 DST>)}, {'text': u'White bean soup was great', 'datetime': datetime.datetime(2017, 7, 31, 19, 40, 28, 546795, tzinfo=<DstTzInfo 'US/Eastern' EDT-1 day, 20:00:00 DST>)}, {'text': u'Fresh peach was not edible ', 'datetime': datetime.datetime(2017, 7, 31, 19, 40, 51, 552736, tzinfo=<DstTzInfo 'US/Eastern' EDT-1 day, 20:00:00 DST>)}, {'text': u'Double espresso was perfect', 'datetime': datetime.datetime(2017, 7, 31, 19, 41, 1, 555391, tzinfo=<DstTzInfo 'US/Eastern' EDT-1 day, 20:00:00 DST>)}]
Tuesday, August 8 [{'text': u'2nd day in a row of 

In [163]:
for channel_name, comments in channel_comments.items():
    personal_messages = [msg for msg in reversed(comments) if msg['type'] == 'message' and 'subtype' not in msg]
    localized_chats = [
        {
            'datetime': (datetime
            .fromtimestamp(float(msg['ts']), timezone('UTC'))
            .astimezone(timezone('US/Eastern'))),
            'text': msg['text']
        }
        for msg in personal_messages
    ]
    
    display(Markdown("## \#%s" % channel_name))

    for day in groupby(localized_chats, lambda x: custom_strftime("%A, %B {S}", x['datetime'])):
        display(Markdown("### %s" % day[0]))

        for message in day[1]:
            display(Markdown("&nbsp;&nbsp; *%s*: %s" % (custom_strftime("%-I:%M %p", message['datetime']), message['text']) ))

        # display(Markdown("<br />"))

# display(Markdown("<br />".join(["**%s:**<br />&nbsp;&nbsp;%s" % (custom_strftime("%A, %B {S}, %-I:%M%p", msg['datetime']), escape_markdown(msg['text'])) for msg in formatted_chats])))

## \#building_comments

### Thursday, August 3rd

&nbsp;&nbsp; *3:47 PM*: Maybe <@U6AKZKWBY> shouldn’t be part of this channel.

&nbsp;&nbsp; *3:47 PM*: Anyway, WiFi (eduroam) in 2B today was on the edge of being unusable.

&nbsp;&nbsp; *3:49 PM*: sorry I am trying to remove it

...

&nbsp;&nbsp; *3:48 PM*: I had a good fruit roll-up today.

### Friday, August 4th

&nbsp;&nbsp; *10:19 AM*: There be hot sauce now! :pray:

&nbsp;&nbsp; *10:23 AM*: Bagel everything was good

&nbsp;&nbsp; *10:23 AM*: Soups are generally very good